# Code To generate lighting for images by using heatmaps

In [1]:
import os

# Model
%env CUDA_VISIBLE_DEVICES=3,2
#%env PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:32

# HF Cache
os.environ["HF_HOME"] = "../.cache"
!echo $HF_HOME
!huggingface-cli whoami

env: CUDA_VISIBLE_DEVICES=3,2
../.cache
TorgeSchwark
orgs:  Basket-AEye


In [2]:
import os
import cv2
import torch
import numpy as np
from PIL import Image
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
import cv2
import os
import torch
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

# Setze die Pfade huggingface/ai_shelf/sd/10classes_without_background
image_path = "../huggingface/ai_shelf/sd/10classes_without_background"       # ← anpassen
output_path = "../huggingface/ai_shelf/sd/test_depth_map"     # ← anpassen

# MiDaS vorbereiten
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_type = "MiDaS_small"
torch.hub.load("intel-isl/MiDaS", "MiDaS_small")
# Modell laden
model = torch.hub.load("intel-isl/MiDaS", model_type)
model.to(device)
model.eval()

# Transformationen laden
midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms")
transform = midas_transforms.small_transform

# Unterstützte Bildformate
image_extensions = {".png", ".jpg", ".jpeg", ".bmp"}

# Alle Bilder rekursiv durchgehen
for root, _, files in os.walk(image_path):
    for file in files:
        ext = os.path.splitext(file)[1].lower()
        if ext not in image_extensions:
            continue

        # Eingabe- und Ausgabepfad vorbereiten
        input_file_path = os.path.join(root, file)
        rel_path = os.path.relpath(root, image_path)
        output_dir = os.path.join(output_path, rel_path)
        os.makedirs(output_dir, exist_ok=True)

        output_file_name = os.path.splitext(file)[0] + "_depth.png"
        output_file_path = os.path.join(output_dir, output_file_name)

        # Bild laden und transformieren
        img = Image.open(input_file_path).convert("RGB")
        input_tensor = transform(img).to(device)

        with torch.no_grad():
            prediction = model(input_tensor.unsqueeze(0))
            prediction = torch.nn.functional.interpolate(
                prediction.unsqueeze(1),
                size=img.size[::-1],  # (H, W)
                mode="bicubic",
                align_corners=False
            ).squeeze()

        depth_map = prediction.cpu().numpy()

        # Normalisieren und speichern
        depth_min = depth_map.min()
        depth_max = depth_map.max()
        if depth_max - depth_min > 1e-6:
            depth_map = (depth_map - depth_min) / (depth_max - depth_min)
        else:
            depth_map = np.zeros_like(depth_map)

        depth_map_uint8 = (depth_map * 255).astype(np.uint8)
        cv2.imwrite(output_file_path, depth_map_uint8)
        print(f"Gespeichert: {output_file_path}")


Using cache found in /home/stu236894/.cache/torch/hub/intel-isl_MiDaS_master


ModuleNotFoundError: No module named 'pymodbus.client.asynchronous'